In [1]:
from importlib import reload
import numpy as np
import os
import pandas as pd
import pygformula
from pygformula import ParametricGformula
from pygformula.parametric_gformula.interventions import static
from pygformula.data import load_basicdata_nocomp
import pickle
import gc
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.formula.api as smf
import statsmodels.api as sm


### G-Formula

In [2]:
data = pd.read_csv('included_cohort_prep.csv')

/tmp/ipykernel_1494396/2382593605.py:1: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('included_cohort_prep.csv')


In [3]:
data.drop(['Unnamed: 0'], inplace=True, axis=1)

In [4]:
data = data.sample(frac=0.1, random_state=1).copy()

In [5]:
data.shape

(749906, 87)

In [6]:
data.columns

Index(['NIA', 'sexe', 'data_naixement', 'abs_c', 'abs', 'pais_c',
       'N_vaccine_total', 'VACUNA_1_DATA', 'VACUNA_1_MOTIU', 'VACUNA_2_DATA',
       'VACUNA_2_MOTIU', 'VACUNA_3_DATA', 'VACUNA_3_MOTIU', 'VACUNA_1_DATA_pp',
       'VACUNA_2_DATA_pp', 'VACUNA_3_DATA_pp', 'DATA_DM_min', 'DM',
       'covid_bef_vax', 'test_date_covid_1', 'test_res_covid_1',
       'test_date_covid_2', 'test_res_covid_2', 'test_date_covid_3',
       'test_res_covid_3', 'test_date_imc_1', 'test_res_imc_1',
       'test_date_imc_2', 'test_res_imc_2', 'test_date_imc_3',
       'test_res_imc_3', 'test_date_sp_1', 'test_res_sp_1', 'test_date_sp_2',
       'test_res_sp_2', 'test_date_sp_3', 'test_res_sp_3', 'test_date_dp_1',
       'test_res_dp_1', 'test_date_dp_2', 'test_res_dp_2', 'test_date_dp_3',
       'test_res_dp_3', 'test_date_abdo_1', 'test_res_abdo_1',
       'test_date_abdo_2', 'test_res_abdo_2', 'test_date_abdo_3',
       'test_res_abdo_3', 'test_date_bg_1', 'test_res_bg_1', 'test_date_bg_2',
       

In [7]:
data.sexe = data.sexe.map({'H':0, 'D':1})

In [8]:
###### CAMBIAR FORMATO DE LOS DATOS (PIVOT) 

In [9]:
for i in range(1,4):
  print(i)
  data.rename({'VACUNA_{}_DATA'.format(i): 'VACUNA_DATA_{}'.format(i)}, axis=1, inplace=True)
  data.rename({'VACUNA_{}_MOTIU'.format(i): 'VACUNA_MOTIU_{}'.format(i)}, axis=1, inplace=True)
  data.rename({'VACUNA_{}_DATA_pp'.format(i): 'VACUNA_DATA_pp_{}'.format(i)}, axis=1, inplace=True)

1
2
3


In [10]:
stubnames = ['VACUNA_DATA_1', 'VACUNA_MOTIU_1', 'VACUNA_DATA_2',
'VACUNA_MOTIU_2', 'VACUNA_DATA_3', 'VACUNA_MOTIU_3', 'VACUNA_DATA_pp_1',
'VACUNA_DATA_pp_2', 'VACUNA_DATA_pp_3', 
'test_date_covid_1', 'test_res_covid_1',
'test_date_covid_2', 'test_res_covid_2', 'test_date_covid_3',
'test_res_covid_3', 'test_date_imc_1', 'test_res_imc_1',
'test_date_imc_2', 'test_res_imc_2', 'test_date_imc_3',
'test_res_imc_3', 'test_date_sp_1', 'test_res_sp_1', 'test_date_sp_2',
'test_res_sp_2', 'test_date_sp_3', 'test_res_sp_3', 'test_date_dp_1',
'test_res_dp_1', 'test_date_dp_2', 'test_res_dp_2', 'test_date_dp_3',
'test_res_dp_3', 'test_date_abdo_1', 'test_res_abdo_1',
'test_date_abdo_2', 'test_res_abdo_2', 'test_date_abdo_3',
'test_res_abdo_3', 'test_date_bg_1', 'test_res_bg_1', 'test_date_bg_2',
'test_res_bg_2', 'test_date_bg_3', 'test_res_bg_3', 'test_date_chol_1',
'test_res_chol_1', 'test_date_chol_2', 'test_res_chol_2',
'test_date_chol_3', 'test_res_chol_3', 'test_date_smoking_1',
'test_res_smoking_1', 'test_date_smoking_2', 'test_res_smoking_2',
'test_date_smoking_3', 'test_res_smoking_3', 'test_date_gma_1',
'test_res_gma_1', 'test_date_gma_2', 'test_res_gma_2',
'test_date_gma_3', 'test_res_gma_3', 'Vacuna_1', 'Vacuna_2', 'Vacuna_3']

In [11]:
for i in range(0,len(stubnames)):
  stubnames[i] = stubnames[i][0:-1]

In [12]:
data_piv = pd.wide_to_long(data, list(set(stubnames)), i='NIA', j='time')
data_piv.reset_index(inplace=True, drop=False)

In [13]:
data_piv.time = data_piv.time - 1

In [14]:
# Transform outcome for binary eof

In [15]:
data_piv.loc[(data_piv.time==1)|(data_piv.time==2), 'DM'] = np.NaN

In [16]:
# G-formula

In [17]:
time_name = 'time'
id = 'NIA'
time_points = np.max(np.unique(data_piv[time_name])) + 1

In [18]:
covnames = [
  'test_res_sp_',
  'test_res_smoking_',
  'test_res_chol_',
  'test_res_abdo_',
  'test_res_dp_',
  'test_res_imc_',
  'test_res_bg_',
  'test_res_covid_',
  'test_res_gma_',
  'Vacuna_']

covtypes = [
  'normal',
  'categorical',
  'normal',
  'normal',
  'normal',
  'normal',
  'normal',
  'categorical',
  'truncated normal',
  'binary']

trunc_params = ['NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'NA', [0, 'left'], 'NA']

covmodels = [
  'test_res_sp_ ~ lag1_test_res_sp_ + test_res_imc_ + data_naixement + pais_c + C(test_res_smoking_) + test_res_sociostat_1+ time',
  'test_res_smoking_ ~ C(lag1_test_res_smoking_) + data_naixement + pais_c + test_res_sociostat_1+ time',
  'test_res_chol_ ~ lag1_test_res_chol_ + data_naixement + pais_c + test_res_sociostat_1+ time',
  'test_res_abdo_ ~ lag1_test_res_abdo_ + test_res_imc_ +  + data_naixement + pais_c + C(test_res_smoking_) + time',
  'test_res_dp_ ~ lag1_test_res_dp_ + test_res_imc_ + data_naixement + pais_c + C(test_res_smoking_) + test_res_sociostat_1+ time',
  'test_res_imc_ ~ lag1_test_res_imc_ + data_naixement + pais_c + C(test_res_smoking_) + test_res_sociostat_1+ time',
  'test_res_bg_ ~ lag1_test_res_bg_ + test_res_imc_ + data_naixement + pais_c + test_res_sociostat_1+ time',
  'test_res_covid_ ~ C(lag1_test_res_covid_) + test_res_imc_ + data_naixement + pais_c + test_res_bg_ + C(test_res_smoking_) + test_res_sociostat_1+ time',
  'test_res_gma_ ~ lag1_test_res_gma_ + test_res_imc_ + data_naixement + test_res_dp_ + test_res_sp_ + test_res_chol_ + pais_c + test_res_bg_ + C(test_res_smoking_) + time',
  'Vacuna_ ~ lag1_Vacuna_ + time']

basecovs = ['abs_c', 'pais_c', 'sexe', 'data_naixement', 'test_res_sociostat_1'] 

outcome_name = 'DM'
ymodel = 'DM ~ abs_c + pais_c + sexe + data_naixement + test_res_sociostat_1+\
  test_res_sp_ + lag1_test_res_sp_ +\
  C(test_res_smoking_) + C(lag1_test_res_smoking_)+\
  test_res_chol_ + lag1_test_res_chol_+\
  test_res_abdo_ + lag1_test_res_abdo_+\
  test_res_dp_ + lag1_test_res_dp_+\
  test_res_imc_ + lag1_test_res_imc_+\
  test_res_bg_ + lag1_test_res_bg_+\
  C(test_res_covid_) + C(lag1_test_res_covid_)+\
  test_res_gma_ + lag1_test_res_gma_'
 
outcome_type = 'binary_eof'
int_descript = ['Never treat', 'Treat on Vacuna only at t1', 'Treat on Vacuna only at t1 & t2', 'Treat on Vacuna at t1, t2 & t3']
ymodel_type = None

Intervention1_Vacuna = [static, np.zeros(time_points)]
Intervention2_Vacuna = [static, np.ones(time_points), [0]]
Intervention3_Vacuna = [static, np.ones(time_points), [0, 1]]
Intervention4_Vacuna = [static, np.ones(time_points), [0, 1, 2]]

In [19]:
covnames = [
  'test_res_sp_',
  'test_res_smoking_',
  'test_res_chol_',
  'test_res_abdo_',
  'test_res_dp_',
  'test_res_imc_',
  'test_res_bg_',
  'test_res_covid_',
  'test_res_gma_',
  'Vacuna_']

covtypes = [
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown',
  'unknown']

trunc_params = ['NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'NA', 'NA', 'NA']

covmodels = [
  'test_res_sp_ ~ lag1_test_res_sp_ + test_res_imc_ + data_naixement + pais_c + test_res_smoking_ + test_res_sociostat_1 + time',
  'test_res_smoking_ ~ lag1_test_res_smoking_ + data_naixement + pais_c + test_res_sociostat_1 + time',
  'test_res_chol_ ~ lag1_test_res_chol_ + data_naixement + pais_c + test_res_sociostat_1 + time',
  'test_res_abdo_ ~ lag1_test_res_abdo_ + test_res_imc_ + data_naixement + pais_c + test_res_smoking_ + time',
  'test_res_dp_ ~ lag1_test_res_dp_ + test_res_imc_ + data_naixement + pais_c + test_res_smoking_ + test_res_sociostat_1 + time',
  'test_res_imc_ ~ lag1_test_res_imc_ + data_naixement + pais_c + test_res_smoking_ + test_res_sociostat_1 + time',
  'test_res_bg_ ~ lag1_test_res_bg_ + test_res_imc_ + data_naixement + pais_c + test_res_sociostat_1 + time',
  'test_res_covid_ ~ lag1_test_res_covid_ + test_res_imc_ + data_naixement + pais_c + test_res_bg_ + test_res_smoking_ + test_res_sociostat_1 + time',
  'test_res_gma_ ~ lag1_test_res_gma_ + test_res_imc_ + data_naixement + test_res_dp_ + test_res_sp_ + test_res_chol_ + pais_c + test_res_bg_ + test_res_smoking_ + time',
  'Vacuna_ ~ lag1_Vacuna_ + time']

basecovs = ['abs_c', 'pais_c', 'sexe', 'data_naixement', 'test_res_sociostat_1'] 

outcome_name = 'DM'
ymodel = 'DM ~ abs_c + pais_c + sexe + data_naixement + test_res_sociostat_1+\
  test_res_sp_ + lag1_test_res_sp_ +\
  test_res_smoking_ + lag1_test_res_smoking_+\
  test_res_chol_ + lag1_test_res_chol_+\
  test_res_abdo_ + lag1_test_res_abdo_+\
  test_res_dp_ + lag1_test_res_dp_+\
  test_res_imc_ + lag1_test_res_imc_+\
  test_res_bg_ + lag1_test_res_bg_+\
  test_res_covid_ + lag1_test_res_covid_+\
  test_res_gma_ + lag1_test_res_gma_'
 
outcome_type = 'binary_eof'
int_descript = ['Never treat', 'Treat on Vacuna only at t1', 'Treat on Vacuna only at t1 & t2', 'Treat on Vacuna at t1, t2 & t3']
ymodel_type = 'Random_forest'

Intervention1_Vacuna = [static, np.zeros(time_points)]
Intervention2_Vacuna = [static, np.ones(time_points), [0]]
Intervention3_Vacuna = [static, np.ones(time_points), [0, 1]]
Intervention4_Vacuna = [static, np.ones(time_points), [0, 1, 2]]

In [20]:
float64_cols = list(data_piv.select_dtypes(include='float64'))

# The same code again calling the columns
data_piv[float64_cols] = data_piv[float64_cols].astype('float32')

In [21]:
gc.collect()

0

In [22]:
#%%capture cap
#%%time
print('********************************************************************')
g = ParametricGformula(obs_data = data_piv, id = id, time_name=time_name,
             time_points = time_points, int_descript = int_descript,
             covnames=covnames, covtypes=covtypes, trunc_params=trunc_params,
             covmodels=covmodels, basecovs=basecovs,
             outcome_name=outcome_name, ymodel=ymodel, ymodel_type=ymodel_type, outcome_type=outcome_type,
             Intervention1_Vacuna = Intervention1_Vacuna,
             Intervention2_Vacuna = Intervention2_Vacuna,
             Intervention3_Vacuna = Intervention3_Vacuna,
             Intervention4_Vacuna = Intervention4_Vacuna,
             parallel=True, ncores=18)

g.fit()
print('********************************************************************')

********************************************************************
start fitting parametric model.
Using RF for outcome model
start simulating.


PicklingError: Could not pickle the task to send it to the workers.

In [21]:
# Save the captured output to a text file
with open('outputRF.txt', 'a') as file:
    file.write(cap.stdout)

In [22]:
# Serialize the object to a binary format
with open('gformRF.pkl', 'wb') as file:
    pickle.dump(g, file)

In [ ]:
g.plot_natural_course()

In [10]:
# Deserialize the object from the binary file
#with open('gform.pkl', 'rb') as file:
#    g = pickle.load(file)

### Notes:
- Time has to start at 0, otherwise we get errors of type "operands of shape () could not be broadcast together"
- No NaNs in the covariates, otherwise we get errors of type "operands of shape () could not be broadcast together"

### Full version with real data

In [ ]:
### Investigate other intervention options (grace period etc.)

### OTHERS

In [26]:
# Set seed for reproducibility
np.random.seed(123)

# Create a data frame with random values
data = pd.DataFrame({
    'Profit': np.random.uniform(1000, 5000, 100),  # Random values between 1000 and 5000
    'Sugar': np.random.uniform(0, 100, 100),       # Random values between 0 and 100
    'Power': np.random.uniform(10, 500, 100),      # Random values between 10 and 500
    'Women': np.random.randint(0, 2, 100)          # Random 0s and 1s
})


class DataSet(dict):
    def __init__(self, path):
        self.parquet = pq.ParquetFile(path)

    def __getitem__(self, key):
        try:
            return self.parquet.read([key]).to_pandas()[key]
        except:
            raise KeyError

In [25]:
# Write the data to a CSV file
data.to_parquet('random_data.parquet')

# Display the first few rows of the data frame
data.head()

# Save data_parquet other way
data_parquet = data.to_parquet(index=False)

In [25]:
pa.parquet.read_schema('temp.parquet', memory_map=True)

NIA: int64
time: int64
DATA_DM_min: string
DM: float
ISC reescalat: float
N_vaccine_total: int64
abs: string
abs_c: float
age_1: float
age_2: float
age_3: float
covid_bef_vax: int64
data_naixement: int64
idabs: float
pais_c: float
sexe: string
test_date_sociostat_1: string
test_date_sociostat_2: string
test_date_sociostat_3: string
test_res_sociostat_1: float
test_res_sociostat_2: float
test_res_sociostat_3: float
test_res_covid_: float
test_date_abdo_: string
test_date_smoking_: string
test_res_bg_: float
VACUNA_MOTIU_: string
test_res_dp_: float
VACUNA_DATA_pp_: string
test_date_gma_: string
test_date_chol_: string
test_res_abdo_: float
VACUNA_DATA_: string
test_date_sp_: string
test_res_sp_: float
test_res_imc_: float
test_res_gma_: float
test_date_covid_: string
test_date_dp_: string
Vacuna_: float
test_date_imc_: string
test_res_smoking_: float
test_date_bg_: string
test_res_chol_: float
lag1_test_res_sp_: float
lag1_test_res_smoking_: float
lag1_test_res_chol_: float
lag1_test_re

In [27]:
LargeData = DataSet('temp.parquet')

#res = smf.ols('test_res_sp_ ~ lag1_test_res_sp_ + test_res_imc_ + data_naixement + pais_c + C(test_res_smoking_) + test_res_sociostat_1+ time', data=LargeData).fit()
smf.glm(covmodels[0], data=LargeData, family=sm.families.Gaussian()).fit()

MemoryError: Unable to allocate 146. GiB for an array with shape (1499812, 13027) and data type float64

In [42]:
LargeData = DataSet('random_data.parquet')

res = smf.ols('Profit ~ Sugar + Power', data=LargeData).fit()

In [28]:
LargeData = DataSet(data_parquet)

res = smf.ols('Profit ~ Sugar + Power', data=LargeData).fit()

TypeError: Cannot convert bytes to pyarrow.lib.NativeFile

In [38]:
DataSet(data_parquet)

TypeError: Cannot convert bytes to pyarrow.lib.NativeFile